In [1]:
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from numpy import asarray
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
import cv2

In [2]:
train_dir=r'C:\Users\HP\Desktop\Trafik_isaretler_dataset\Train'
test_dir=r'C:\Users\HP\Desktop\Trafik_isaretler_dataset\Test'

In [3]:
classes = [ 'Speed limit (20km/h)',
            'Speed limit (30km/h)', 
            'Speed limit (50km/h)', 
            'Speed limit (60km/h)', 
            'Speed limit (70km/h)', 
            'Speed limit (80km/h)', 
            'End of speed limit (80km/h)', 
            'Speed limit (100km/h)', 
            'Speed limit (120km/h)', 
            'No passing', 
            'No passing veh over 3.5 tons', 
            'Right-of-way at intersection', 
            'Priority road', 
            'Yield', 
            'Stop', 
            'No vehicles', 
            'Veh > 3.5 tons prohibited', 
            'No entry', 
            'General caution', 
            'Dangerous curve left', 
            'Dangerous curve right', 
            'Double curve', 
            'Bumpy road', 
            'Slippery road', 
            'Road narrows on the right', 
            'Road work', 
            'Traffic signals', 
            'Pedestrians', 
            'Children crossing', 
            'Bicycles crossing', 
            'Beware of ice/snow',
            'Wild animals crossing', 
            'End speed + passing limits', 
            'Turn right ahead', 
            'Turn left ahead', 
            'Ahead only', 
            'Go straight or right', 
            'Go straight or left', 
            'Keep right', 
            'Keep left', 
            'Roundabout mandatory', 
            'End of no passing', 
            'End no passing veh > 3.5 tons' ]

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255 ,validation_split=0.2)

In [5]:
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    color_mode='rgb',
    batch_size=16,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)
validation_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    color_mode='rgb',
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 31369 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [6]:
train_generator.class_indices

{'0': 0,
 '1': 1,
 '10': 2,
 '11': 3,
 '12': 4,
 '13': 5,
 '14': 6,
 '15': 7,
 '16': 8,
 '17': 9,
 '18': 10,
 '19': 11,
 '2': 12,
 '20': 13,
 '21': 14,
 '22': 15,
 '23': 16,
 '24': 17,
 '25': 18,
 '26': 19,
 '27': 20,
 '28': 21,
 '29': 22,
 '3': 23,
 '30': 24,
 '31': 25,
 '32': 26,
 '33': 27,
 '34': 28,
 '35': 29,
 '36': 30,
 '37': 31,
 '38': 32,
 '39': 33,
 '4': 34,
 '40': 35,
 '41': 36,
 '42': 37,
 '5': 38,
 '6': 39,
 '7': 40,
 '8': 41,
 '9': 42}

In [7]:
test_generator=ImageDataGenerator(rescale=.1/255).flow_from_directory(
    test_dir,
    target_size=(100,100),
    color_mode='rgb',
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 12629 images belonging to 1 classes.


In [8]:
batchX  = test_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
type(batchX[0])
batchX.dtype
batchX[0]

Batch shape=(1, 100, 100, 3), min=0.001, max=0.075


array([[[0.04509804, 0.05411765, 0.06784314],
        [0.04509804, 0.05411765, 0.06784314],
        [0.04549019, 0.05411765, 0.06745098],
        ...,
        [0.03137255, 0.03960784, 0.04941176],
        [0.02470588, 0.03215686, 0.04117647],
        [0.02470588, 0.03215686, 0.04117647]],

       [[0.04509804, 0.05411765, 0.06784314],
        [0.04509804, 0.05411765, 0.06784314],
        [0.04549019, 0.05411765, 0.06745098],
        ...,
        [0.03137255, 0.03960784, 0.04941176],
        [0.02470588, 0.03215686, 0.04117647],
        [0.02470588, 0.03215686, 0.04117647]],

       [[0.04627451, 0.05607843, 0.06980392],
        [0.04627451, 0.05607843, 0.06980392],
        [0.04549019, 0.05490196, 0.06941176],
        ...,
        [0.04666667, 0.0545098 , 0.06549019],
        [0.04588235, 0.05333333, 0.06509804],
        [0.04588235, 0.05333333, 0.06509804]],

       ...,

       [[0.04470588, 0.05294118, 0.06549019],
        [0.04470588, 0.05294118, 0.06549019],
        [0.04470588, 0

In [13]:
model=Sequential()
model.add(Conv2D(32, kernel_size=(5,5), activation='relu',input_shape=(100,100,3)))
model.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(43,activation='softmax'))

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
1960/1960 [==============================] - 272s 138ms/step - loss: 1.8605 - accuracy: 0.4501 - val_loss: 0.8099 - val_accuracy: 0.8083
Epoch 2/20
1960/1960 [==============================] - 271s 138ms/step - loss: 1.0332 - accuracy: 0.6523 - val_loss: 0.5263 - val_accuracy: 0.8624
Epoch 3/20
1960/1960 [==============================] - 270s 138ms/step - loss: 0.8529 - accuracy: 0.7042 - val_loss: 0.4114 - val_accuracy: 0.8902
Epoch 4/20
1960/1960 [==============================] - 270s 138ms/step - loss: 0.7670 - accuracy: 0.7318 - val_loss: 0.3316 - val_accuracy: 0.9162
Epoch 5/20
1960/1960 [==============================] - 267s 136ms/step - loss: 0.6903 - accuracy: 0.7586 - val_loss: 0.3174 - val_accuracy: 0.9168
Epoch 6/20
1960/1960 [==============================] - 266s 136ms/step - loss: 0.6229 - accuracy: 0.7807 - val_loss: 0.2917 - val_accuracy: 0.9277
Epoch 7/20
1960/1960 [==============================] - 266s 136ms/step - loss: 0.6015 - accuracy: 0.7875 - val_

In [16]:
model.evaluate(validation_generator,
steps=STEP_SIZE_VALID)

490/490 [==============================] - 23s 46ms/step - loss: 0.2008 - accuracy: 0.9462


[0.2008277028799057, 0.9461734890937805]

In [17]:
model.save("trafik_model_43class_100x100.h5")

In [31]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels.items())

dict_items([(0, '0'), (1, '1'), (2, '10'), (3, '11'), (4, '12'), (5, '13'), (6, '14'), (7, '15'), (8, '16'), (9, '17'), (10, '18'), (11, '19'), (12, '2'), (13, '20'), (14, '21'), (15, '22'), (16, '23'), (17, '24'), (18, '25'), (19, '26'), (20, '27'), (21, '28'), (22, '29'), (23, '3'), (24, '30'), (25, '31'), (26, '32'), (27, '33'), (28, '34'), (29, '35'), (30, '36'), (31, '37'), (32, '38'), (33, '39'), (34, '4'), (35, '40'), (36, '41'), (37, '42'), (38, '5'), (39, '6'), (40, '7'), (41, '8'), (42, '9')])


In [21]:
file_name=r"C:\Users\HP\.spyder-py3\imgsign.png"
img=cv2.imread(file_name)
img=tf.expand_dims(img,axis=0)
pred=model.predict(img)

ValueError: in user code:

    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\HP\anaconda3\envs\tenserflow\lib\site-packages\keras\engine\input_spec.py", line 295

In [43]:
np.argmax(pred)

32

In [ ]:
yuklenen_model=models.load_model("trafik_model_yeni.h5")